In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch 

In [2]:
from collections import defaultdict

In [3]:
#login to huggingface hub
from huggingface_hub import login
login()

In [4]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

In [5]:
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16, use_auth_token=True)

Loading model and tokenizer...


/scratch/aseth7/conda-envs/dm-proj/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


/scratch/aseth7/conda-envs/dm-proj/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Step 2: Create a text-generation pipeline
print("Creating text-generation pipeline...")
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=device
)


Creating text-generation pipeline...


In [8]:
terminators = [
tokenizer.eos_token_id,
tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [34]:
print("Loading MMLU dataset...")
dataset = load_dataset("cais/mmlu", "all")

Loading MMLU dataset...


In [10]:
# def evaluate_on_mmlu(model_pipeline, dataset, num_samples=None, batch_size=8):
#     total_correct = 0
#     total_questions = 0
#     subject_results = defaultdict(lambda: {"correct": 0, "total": 0})
#     csv_results = []

#     print("Evaluating on MMLU...")
#     # Limit dataset size if num_samples is provided
#     if num_samples:
#         dataset['test'] = dataset['test'].select(range(num_samples))

#     for batch_start in range(0, len(dataset['test']), batch_size):
#         batch = dataset['test'].select(range(batch_start, min(batch_start + batch_size, len(dataset['test']))))
#         inputs = []
#         print(f"Processing batch {batch_start + 1} - {batch_start + len(batch)}...")
#         for i in range(len(batch)):
#             question = batch["question"][i]
#             choices = batch["choices"][i]
#             correct_answer = batch["answer"][i]
#             subject = batch["subject"][i]

#             prompt = (
#                 f"Question: {question}\n"
#                 f"Choices: {', '.join(choices)}\n"
#                 "Think step-by-step. Please provide your answer in the format 'Steps: [steps]; Answer: [choice]'."
#             )
#             inputs.append(prompt)

#         # Generate predictions in batch
#         outputs = model_pipeline(
#             inputs, 
#             max_length=2048, 
#             num_return_sequences=1,
#             return_full_text=False
#         )

#         for i, output in enumerate(outputs):
#             question = batch["question"][i]
#             choices = batch["choices"][i]
#             correct_choice = choices[batch["answer"][i]]
#             subject = batch["subject"][i]
#             print(f"\nQuestion: {question}")
#             print(f"Choices: {', '.join(choices)}")
#             print(f"Correct Answer: {correct_choice}")
#             print(f"Prediction: {output[0]['generated_text']}")

#             # Parse the answer from model output
#             print("DEBUG: ", output)
#             prediction = output[0]["generated_text"]
#             if "Answer:" in prediction:
#                 prediction = prediction.split("Answer:")[-1].strip()
#             else:
#                 prediction = ""

#             # Check correctness
#             is_correct = (prediction == correct_choice)
#             if is_correct:
#                 total_correct += 1
#                 subject_results[subject]["correct"] += 1
#             subject_results[subject]["total"] += 1
#             total_questions += 1

#             # Save question result to CSV results
#             csv_results.append({
#                 "subject": subject,
#                 "question": question,
#                 "choices": choices,
#                 "correct_answer": correct_choice,
#                 "prediction": prediction,
#                 "is_correct": is_correct
#             })

#     # Calculate overall accuracy
#     overall_accuracy = total_correct / total_questions if total_questions > 0 else 0

#     # Print per-subject results
#     print("\nPer-Subject Accuracy:")
#     for subject, result in subject_results.items():
#         subject_accuracy = result["correct"] / result["total"] if result["total"] > 0 else 0
#         print(f"{subject}: {subject_accuracy * 100:.2f}%")

#     # Print overall accuracy
#     print(f"\nOverall Accuracy: {overall_accuracy * 100:.2f}%")

#     return overall_accuracy, subject_results, csv_results

In [41]:
from collections import defaultdict


def generate_prompt(subject, examples, actual_question, actual_choices):
    """
    Generates a prompt with examples followed by the actual question.
    """
    example_str = ""
    for example in examples:
        example_str += (
            f"Example:\n"
            f"Question: {example['question']}\n"
            f"Choices: {example['choices']}\n"
            f"Answer: {example['answer']}\n\n"
        )

    prompt = [
        {"role": "system", "content": "You are a helpful assistant that provides detailed and thoughtful responses."},
        {"role": "user", "content": f"Following is a multiple-choice question on {subject}. Think step-by-step and return the correct answer choice. Return your answer in the format Steps: [steps] \nAnswer: [choice]."},
        {"role": "user", "content": (
            f"{example_str}"
            f"Question: {actual_question}\n"
            f"Choices: {actual_choices}\n"
            f"Answer:"
        )}
    ]
    return prompt


def evaluate_on_mmlu(generator, dataset, num_samples=None, batch_size=8):
    """
    Evaluate the model on the MMLU dataset using the correct prompt structure.
    """
    total_correct = 0
    total_questions = 0
    subject_results = defaultdict(lambda: {"correct": 0, "total": 0})

    print("Evaluating on MMLU...")
    if num_samples:
        dataset['test'] = dataset['test'].select(range(num_samples))

    for batch_start in range(0, len(dataset['test']), batch_size):
        batch = dataset['test'].select(range(batch_start, min(batch_start + batch_size, len(dataset['test']))))
        
        for i in range(len(batch)):
            # Extract details for the current test question
            question = batch["question"][i]
            choices = " ".join([f"{j + 1}. {choice}" for j, choice in enumerate(batch["choices"][i])])
            correct_answer_index = batch["answer"][i]
            subject = batch["subject"][i]
            correct_choice = batch["choices"][i][correct_answer_index]

            # Select examples for the prompt
            examples = dataset["dev"].filter(lambda ex: ex["subject"] == subject)
            examples = [
                {
                    "question": ex["question"],
                    "choices": " ".join([f"{j + 1}. {choice}" for j, choice in enumerate(ex["choices"])]),
                    "answer": f"{ex['answer'] + 1}"  # Convert to 1-based index for display
                }
                for ex in examples
            ][:3]

            # Generate the prompt
            prompt = generate_prompt(subject, examples, question, choices)

            # Generate prediction
            generation = generator(prompt, max_new_tokens=1000, do_sample=False)
            raw_output = generation[0]['generated_text'][3]['content']
            print(f"\nRaw Model Output:\n{raw_output}\n")

            # Extract the model's answer
            if "Answer:" in raw_output:
                prediction = raw_output.split("Answer:")[-1].strip().split("\n")[0].strip()
            else:
                prediction = ""

            # Check correctness
            is_correct = prediction == f"{correct_answer_index + 1}"  # Compare as 1-based index
            total_correct += int(is_correct)
            subject_results[subject]["correct"] += int(is_correct)
            subject_results[subject]["total"] += 1
            total_questions += 1

            print(f"Question: {question}")
            print(f"Choices: {choices}")
            print(f"Prediction: {prediction}")
            print(f"Correct: {correct_choice} (Answer: {correct_answer_index + 1})\n")

    # Calculate overall accuracy
    overall_accuracy = total_correct / total_questions if total_questions > 0 else 0
    print(f"\nOverall Accuracy: {overall_accuracy * 100:.2f}%")

    # Per-subject accuracy
    print("\nPer-Subject Accuracy:")
    for subject, result in subject_results.items():
        accuracy = (result["correct"] / result["total"]) * 100 if result["total"] > 0 else 0
        print(f"{subject}: {accuracy:.2f}%")

    return overall_accuracy, subject_results

In [42]:
overall_accuracy, subject_results = evaluate_on_mmlu(generation_pipeline, dataset, num_samples=5)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Evaluating on MMLU...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Raw Model Output:
Steps: 
1. First, we need to simplify the expression sqrt(18). 
   We can write sqrt(18) as sqrt(9 * 2) = 3 * sqrt(2).
2. Now, we have the field extension Q(sqrt(2), sqrt(3), 3 * sqrt(2)).
3. Since 3 * sqrt(2) is already in the field Q(sqrt(2), sqrt(3)), we don't need to extend the field further.
4. The degree of the field extension is the number of elements in a basis for the extension field over the base field.
5. The basis for Q(sqrt(2), sqrt(3)) over Q is {1, sqrt(2), sqrt(3), sqrt(6)}.
6. The basis for Q(sqrt(2), sqrt(3), 3 * sqrt(2)) over Q(sqrt(2), sqrt(3)) is {1}.
7. Therefore, the degree of the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q is the degree of Q(sqrt(2), sqrt(3)) over Q, which is 4.

Answer: 2

Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
Choices: 1. 0 2. 4 3. 2 4. 6
Prediction: 2
Correct: 4 (Answer: 2)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Raw Model Output:
To find the index of <p> in S_5, we first need to determine the order of the subgroup generated by p.

The given permutation p = (1, 2, 5, 4)(2, 3) can be written as a product of disjoint cycles. 

The cycle (1, 2, 5, 4) has a length of 4, and the cycle (2, 3) has a length of 2. 

Since the lengths of the disjoint cycles are relatively prime (4 and 2 are coprime), the order of the permutation p is the product of the lengths of the disjoint cycles, which is 4 * 2 = 8.

Now, we need to find the index of the subgroup <p> in S_5. The index of a subgroup H in a group G is given by the formula:

|G| / |H|

where |G| is the order of the group G and |H| is the order of the subgroup H.

In this case, the order of S_5 is 5! = 120, and the order of the subgroup <p> is 8.

So, the index of <p> in S_5 is:

|S_5| / |<p>| = 120 / 8 = 15

However, this is not among the given choices. We need to consider the fact that the index of a subgroup is always a divisor of the order of the gr

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Raw Model Output:
To find the zeros of the polynomial x^5 + 3x^3 + x^2 + 2x in Z_5, we need to check each element of Z_5 to see if it is a zero of the polynomial.

Z_5 = {0, 1, 2, 3, 4}

We can plug each element into the polynomial and check if the result is 0.

1. x = 0: 0^5 + 3(0)^3 + 0^2 + 2(0) = 0
2. x = 1: 1^5 + 3(1)^3 + 1^2 + 2(1) = 1 + 3 + 1 + 2 = 7 ≡ 2 (mod 5)
3. x = 2: 2^5 + 3(2)^3 + 2^2 + 2(2) = 32 + 24 + 4 + 4 = 64 ≡ 4 (mod 5)
4. x = 3: 3^5 + 3(3)^3 + 3^2 + 2(3) = 243 + 81 + 9 + 6 = 339 ≡ 4 (mod 5)
5. x = 4: 4^5 + 3(4)^3 + 4^2 + 2(4) = 1024 + 192 + 16 + 8 = 1240 ≡ 0 (mod 5)

So, the zeros of the polynomial in Z_5 are 0 and 4.

Steps: [Check each element of Z_5 to see if it is a zero of the polynomial]
Answer: 4

Question: Find all zeros in the indicated finite field of the given polynomial with coefficients in that field. x^5 + 3x^3 + x^2 + 2x in Z_5
Choices: 1. 0 2. 1 3. 0,1 4. 0,4
Prediction: 4
Correct: 0,4 (Answer: 4)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Raw Model Output:
Steps: 
1. A factor group of a non-Abelian group can be Abelian if the normal subgroup is Abelian.
2. If K is a normal subgroup of H and H is a normal subgroup of G, then K is indeed a normal subgroup of G.
3. Therefore, Statement 1 is False and Statement 2 is True.

Answer: 2

Question: Statement 1 | A factor group of a non-Abelian group is non-Abelian. Statement 2 | If K is a normal subgroup of H and H is a normal subgroup of G, then K is a normal subgroup of G.
Choices: 1. True, True 2. False, False 3. True, False 4. False, True
Prediction: 2
Correct: False, False (Answer: 2)


Raw Model Output:
To find the product of the given polynomials in the given polynomial ring, we need to multiply them.

f(x) = 4x - 5
g(x) = 2x^2 - 4x + 2

First, multiply each term of f(x) by each term of g(x):

(4x)(2x^2) = 8x^3
(4x)(-4x) = -16x^2
(4x)(2) = 8x
(-5)(2x^2) = -10x^2
(-5)(-4x) = 20x
(-5)(2) = -10

Now, combine like terms:

8x^3 - 16x^2 + 8x - 10x^2 + 20x - 10
Combine like ter

In [12]:
for i, sample in enumerate(dataset):
    print(sample)

test
validation
dev
auxiliary_train


In [ ]:
dataset['test'][100]

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}

In [38]:
dataset['test'].select(range(5))[4]

{'question': 'Find the product of the given polynomials in the given polynomial ring. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 in Z_8[x].',
 'subject': 'abstract_algebra',
 'choices': ['2x^2 + 5', '6x^2 + 4x + 6', '0', 'x^2 + 1'],
 'answer': 1}